# Зависимости

In [33]:
import csv
import json
from shortcuts import (read_portion_data, append_functions_in_json, 
                       filter_functions_name, load_functions, 
                       dataset_save_to_csv, load_functions_from_json)
from settings import FUNCTIONS_PATH, FILTERED_FUNCTIONS_PATH, DATASET_PATH

# Выполнение анализа имен функций

### Подсчёт количества функций

In [84]:
with open(FILTERED_FUNCTIONS_PATH, 'r') as json_file:
    fcounter = 0
    for line in json_file:
        fcounter+=1
print('Общее количество функций в датасете:', fcounter)

Общее количество функций в датасете: 165663


### Подсчет уникальных имен функций и значений узлов

Из-за большого размера файлов оперативная память не справиться с обработкой, поэтому производится порциональный анализ функций.

* `batch_size` - размер порции (1000 для ОЗУ объемом 4 Гб)

* `uniq_names` - словарь уникальных имен функций со стчетчиком

* `uniq_values` - словарь уникальных значений узлов функций

In [ ]:
uniq_names = {}
uniq_values = {}

batch_size = 1000
pos = 0 # начальная позиция чтения файла
counter = 0 # счётчик обработанных функций

# Порционный перебор функций из датасета
functions, pos = read_portion_data(FUNCTIONS_PATH, pos, 1)
while functions:
    functions, pos = read_portion_data(FUNCTIONS_PATH, pos, batch_size)
    
    counter += len(functions) 
        
    # Добавить имена в словарь подсчета уникальных имен функций
    for func in functions:
        func = json.loads(func)
        uniq_names[func[0]['value']] = uniq_names.get(func[0]['value'], 0) + 1
        # Добавить имена в словарь подсчета уникальных значений узлов
        for node in func:
            if 'value' in node:
                uniq_values[node['value']] = uniq_values.get(node['value'], 0) + 1
            
    print('Функций обработано: ', counter, '| Ун. имен', len(uniq_names), '| Ун. знач.:', len(uniq_values))

In [44]:
print('Функций обработано: ', counter, '| Ун. имен', len(uniq_names), '| Ун. знач.:', len(uniq_values))
print('Функций потеряно:', fcounter-counter)

Функций обработано:  1273210 | Ун. имен 535819 | Ун. знач.: 2437048
Функций потеряно: 1275


### Выбор самых частотных имен функций

* th - (thresholder) задает порог вхождения имен функциий или значений в выборку

In [60]:
name_th = 400
value_th = 200

In [61]:
def cut_dict(dictionary : dict, thresholder : int) -> set:
    data = []
    for key, count in dictionary.items():
        if count > thresholder:
            data.append(key)
    return set(data)

In [62]:
names = cut_dict(uniq_names, name_th)
values = cut_dict(uniq_values, value_th)

In [63]:
print('Уникальных имен функций: ', len(names))
print('Уникальных значений узлов функций: ', len(values))

Уникальных имен функций:  138
Уникальных значений узлов функций:  10273


### Фильтрация имен функций по значению

В выборку попадают функции, которые сложно классифицировать по их сниппету, поэтому такие имена функций исключаются из выборки.

In [70]:
bad_names = ['main', 'run', 'init',
            'iter', 'enter','exit',
            'hash', 'del', 'start',
            'nonzero', 'func', 'foo', '_run']
names = set(filter(lambda name: name not in bad_names and len(name) > 2, names))

In [71]:
print('Уникальных имен функций после чистки на неподходящие имена: ', len(names))

Уникальных имен функций после чистки на неподходящие имена:  125


### Фильтрация уникальных значений

Избавление от значений не несущих много информации

In [96]:
values = set(filter(lambda value: len(value) > 1, values))

In [98]:
print('Уникальных значений после чистки на неподходящие значения: ', len(values))

Уникальных значений после чистки на неподходящие значения:  10200


## Фильтрация функций по выбранным именам и трансформация узлов

Все функции из файла `functions.json` будут отфильтрованы по имени и записаны в новый файл `filtered_functions.json`.

### Фильтрация по имени

In [83]:
pos = 0
ffcounter = 0

functions, pos = read_portion_data(FUNCTIONS_PATH, pos, 1)
while functions:
    functions, pos = read_portion_data(FUNCTIONS_PATH, pos, batch_size)
    
    for i,function in enumerate(functions):
        functions[i] = json.loads(functions[i])

    filterd_functions = filter_functions_name(functions, names)
    ffcounter += len(filterd_functions)
    
    append_functions_in_json(
        FILTERED_FUNCTIONS_PATH, 
        filterd_functions
    )

In [99]:
print('Количество функций после фильтрации:', fcounter)

Общее количество функций после фильтрации: 165663


### Трансформация узлов

In [88]:
functions = load_functions_from_json(FILTERED_FUNCTIONS_PATH)

#### Извлечение названия функции из функции

Удаление из обучающей выборки названия функции и сохранение в отдельном массиве для дальнейшей валидации.

In [ ]:
for func in functions:
    assert func[0]['type'] == 'FunctionDef'

In [104]:
labels = []
for i, func in enumerate(functions):
    labels.append(functions[i][0]['value'])
    del functions[i][0]

#### Трансформация

In [106]:
for i,function in enumerate(functions):
    for j,node in enumerate(function):
        try:
            functions[i][j] = node['value'] if node['value'] in values else node['type']
        except KeyError:
            functions[i][j] = node['type']

In [107]:
print('Количество функций после трансформации:', len(functions))

Количество функций после трансформации: 165663


In [111]:
assert len(labels) == len(functions)

In [109]:
print(labels[101], functions[101])

delete ['arguments', 'args', 'self', 'name', 'defaults', 'body', 'Expr', 'Str', 'Return', 'Call', 'AttributeLoad', 'self', 'attr', 'name']


In [110]:
ave = sum(len(func) for func in functions) / len(functions)
print('Средняя длина вектора АСД: ' + str(ave))

Средняя длина вектора АСД: 49.947574292388765


# Сохранение финального датасета

Соединение функции с ее меткой (названием)

In [112]:
functions_dataset = [(label, func) for (label, func) in zip(labels, functions)]

Сохранение финального датасета для обучения в `.csv` файл

In [117]:
dataset_save_to_csv(DATASET_PATH, functions_dataset)